In [8]:
import pandas as pd
import numpy as np
import functools
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
import torchtext
import tqdm
import random
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.lm import MLE
from nltk.util import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import re

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
df=pd.read_csv("/kaggle/input/vvvvvvv/train.csv",index_col="ID")

In [10]:
X = df["Tweet"]
y = np.asarray(df[df.columns[1:]])

In [12]:
df_y=pd.DataFrame(y)
df_y.head()

,0,1,2,3,4,5
0,0,0,0,0,0,0
1,0,0,0,1,1,0
2,1,1,0,1,0,0
3,0,0,0,1,0,0
4,1,1,0,0,0,0


In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import collections


# Any results you write to the current directory are saved as output.

In [14]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [15]:
f=['anger', 'disgust', 'fear', 'joy', 'love',
        'sadness']

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
!pip install transformers

In [19]:
import os
import re
import string
import json
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import warnings

In [20]:
##### Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 80
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 1
EPOCHS = 5
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('roberta-large')

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [21]:
target_cols = [col for col in df.columns if col not in ['Tweet', 'ID']]
target_cols

['anger', 'disgust', 'fear', 'joy', 'love', 'sadness']

In [22]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [23]:
class BERTDataset(Dataset):

    def posaddedtext(self,text):
        
        try:
            doc = nlp(text)
            for token in doc:
                text = text + " " + token.tag_
            return text
        except:
            return text
    
    def neraddedtext(self,text):
        try:
            doc = nlp(text)
            for ent in doc.ents:
                text = text + " " + ent.label_
            return text
        except:
            return text

    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Tweet
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.posaddedtext(self.text[index])
        # text = self.text[index]
#         text = self.neraddedtext(self.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

# class BERTtestDataset(Dataset):
#     def posaddedtext(self,text):
        
#         try:
#             doc = nlp(text)
#             for token in doc:
#                 text = text + " " + token.tag_
#             return text
#         except:
#             return text
    
#     def neraddedtext(self,text):
#         try:
#             doc = nlp(text)
#             for ent in doc.ents:
#                 text = text + " " + ent.label_
#             return text
#         except:
#             return text
    
#     def __init__(self, df, tokenizer, max_len):
#         self.df = df
#         self.max_len = max_len
#         self.text = df.Tweet
#         self.tokenizer = tokenizer
       
        
#     def __len__(self):
#         return len(self.df)
    
#     def __getitem__(self, index):
# #         text = self.posaddedtext(self.text[index])
#         text = self.text[index]
#         # text = self.neraddedtext(self.text[index])
#         inputs = self.tokenizer.encode_plus(
#             text,
#             truncation=True,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             padding='max_length',
#             return_token_type_ids=True
#         )
#         ids = inputs['input_ids']
#         mask = inputs['attention_mask']
#         token_type_ids = inputs["token_type_ids"]
#         return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
    
#         }

In [24]:
train_dataset = BERTDataset(df, tokenizer, MAX_LEN)
# valid_dataset = BERTtestDataset(pf, tokenizer, MAX_LEN)

In [25]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=2, shuffle=True, pin_memory=True)
# valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, 
#                           num_workers=2, shuffle=False, pin_memory=True)

In [26]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta1 = AutoModel.from_pretrained('roberta-large')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(1024,6)
    
    def forward(self, ids, mask, token_type_ids):
        _, features1 = self.roberta1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        # _, features2 = self.roberta2(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict = False)
        # last_hidden_states = torch.cat([features1,features2],dim=1)
        output = self.fc(features1)
        return output

model = BERTClass()
model.to(device);

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [28]:
optimizer = AdamW(params =  model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [29]:
def train(epoch):
    torch.cuda.empty_cache()
    model.train()
    for _,data in enumerate(train_loader, 0):
        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        if _%500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()

In [30]:
print(sum(p.numel() for p in model.parameters()))

355365894


In [31]:
for epoch in range(10):
    train(epoch)

Epoch: 0, Loss:  0.6887431144714355
Epoch: 1, Loss:  0.3119134306907654
Epoch: 2, Loss:  0.22144931554794312
Epoch: 3, Loss:  0.1796567440032959
Epoch: 4, Loss:  0.15937280654907227
Epoch: 5, Loss:  0.14243251085281372
Epoch: 6, Loss:  0.12183939665555954
Epoch: 7, Loss:  0.10662993788719177
Epoch: 8, Loss:  0.08544687926769257
Epoch: 9, Loss:  0.07200583815574646


In [32]:
PATH = "last_model.h5"  

torch.save(model.state_dict(),PATH)

In [38]:
loaded_model = BERTClass()
loaded_model.load_state_dict(torch.load(PATH))
loaded_model.to(device)

# loaded_model

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (roberta1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): Laye

In [40]:

x = " Hope you are well Today"
arr = []
arr.append(x)
pf = pd.DataFrame()
pf['Tweet'] = arr

valid_dataset = BERTtestDataset(pf, tokenizer, MAX_LEN)

valid_loader = DataLoader(valid_dataset, batch_size=1, 
                           num_workers=2, shuffle=False, pin_memory=True)



In [41]:
def validation():
    loaded_model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        print(valid_loader)
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            outputs = loaded_model(ids, mask, token_type_ids)
            
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs

In [42]:
outputs = validation()

In [43]:
outputs

[[0.23080958425998688,
  0.19981300830841064,
  0.006467907223850489,
  0.00375077105127275,
  0.0014900919049978256,
  0.0013780620647594333]]

In [44]:
from IPython.display import FileLink
FileLink(r'last_model.h5')

/kaggle/working/last_model.h5